# EDM Genre Classification - Data Gathering

### Import Libraries

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

import pandas as pd


import pickle

### Set Up Spotify Connection

In [2]:
# Credentials
client_id = 'd72a09f875b3492aa6ffcdbe79ca0786'
client_secret = 'bc01c2662e604e12aa3f36d18d669bc0'
username = '1235520406'

# Scope Options:  https://developer.spotify.com/web-api/using-scopes/
scope = 'user-library-read playlist-modify-public playlist-read-private'

# This can be any valid url, but must be matched on your developer dashboard
redirect_uri = 'http://www.example.com'

def sign_in():
    client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret) 
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

    if token:
        return spotipy.Spotify(auth=token)
    else:
        print("Can't get token for", username)
        
sp = sign_in()

### Get Audio Features Data

In [595]:
def get_recommendations(genre, num_songs, max_iter, sp):
    """
    Generate a set of track URI's (Uniform Resource Indicator) for a given genre.
    Use the recommendations function to generate new lists of 100 songs (the upper
    limit) until the desired number of songs is found or you reach the maximum
    iterations specified. Return as list.
    """
    
    sp = sign_in() # Establish Spotify connection
    
    track_list = set()
    
    i = 1

    while (len(track_list) < num_songs) & (i < max_iter):
        new_tracks = sp.recommendations(seed_genres=[genre], limit=100)['tracks']

        for track in new_tracks:
            track_list.add(track['uri'])
        
        i += 1
    
    print(genre + ': ' + str(len(track_list))) # Output number of tracks obtained
    
    return list(track_list)

In [342]:
def tracks_from_playlists(playlist_uris, sp):
    """
    Return all track URI's for tracks in a given list of playlists by playlist URI.
    """
    
    sp = sign_in()
    
    all_uris = []
    
    for uri in playlist_uris:
        
        results = sp.user_playlist_tracks(playlist_id=uri)
        playlist_tracks = results['items']

        # Loop to ensure all tracks are captured
        while results['next']:
            results = sp.next(results)
            playlist_tracks.extend(results['items'])
        
        # Save playlist track URI's to list
        track_uris = []
        
        for track in playlist_tracks:
            # Skip tracks that have no data
            try:
                track_uri = track['track']['uri']
                track_uris.append(track_uri)
            except TypeError:
                pass
        
        # Append playlist track URI's to full list
        all_uris.extend(track_uris)
    
    return list(set(all_uris))

In [346]:
def get_audio_features(track_list, sp):
    """
    Accept track list and return dataframe with audio features.
    """
    
    sp = sign_in() # Establish Spotify connection
    
    track_list = list(track_list)
    
    audio_df = pd.DataFrame(sp.audio_features(track_list[0]))
    start = 1
    while start < len(track_list):
        end = min(start + 100, len(track_list))
        
        # For subsets with no audio data, loop through one by one and pass on tracks missing data
        try:
            audio_df = pd.concat([audio_df, pd.DataFrame(sp.audio_features(track_list[start:end]))])
        except AttributeError:
            for i in range(start, end):
                try:
                    audio_df = pd.concat([audio_df, pd.DataFrame(sp.audio_features(track_list[i]))])
                except AttributeError:
                    pass
            
        start += 100
    
    return audio_df.reset_index(drop=True)

#### User Generated Playlists

In [136]:
playlists_deep_house = ['spotify:playlist:6vDGVr652ztNWKZuHvsFvx',
                        'spotify:playlist:37i9dQZF1DX2TRYkJECvfC',
                        'spotify:playlist:37i9dQZF1DX5xiztvBdlUf',
                        'spotify:playlist:4wjWoRltrA5qZ0uNsnzdWp',
                        'spotify:playlist:4UJaDEATfh5FoNFyVcMBHV',
                        'spotify:playlist:2GOq4YducOjWm88a4IkCQq',
                        'spotify:playlist:5h1oEk4W9KVMHkOd8WWWlC',
                        'spotify:playlist:3oRNodhtGLVnZl0Q32FJHB',
                        'spotify:playlist:7qKZ5hzM099puSI6MKiUyX',
                        'spotify:playlist:6n7XGSMHS4Xhv0NWL88in9',
                        'spotify:playlist:0ekUBqyN6aMDI90qsn96Gz',
                        'spotify:playlist:0e7ZClzZLXcgtRvhdmfIID',
                        'spotify:playlist:2t5U3e6mHq4T2fKXknS4dd',
                        'spotify:playlist:0ypuLGVb7lGs0WGu8a9PHg',
                        'spotify:playlist:5CLCReqqp7OTLDMcvB1oLw',
                        'spotify:playlist:2b1gr7NkoWDHatEQvhGsjy',
                        'spotify:playlist:3dHfJPYowP6ZHUixqA4pUK',
                        'spotify:playlist:5fwNQf619fTX4jcK7GlJ4l',
                        'spotify:playlist:6r0LRTsaL5DQoZkwheYiw6',
                        'spotify:playlist:0zWcfPRGU4GI2sQSIuwu4j',
                        'spotify:playlist:2J8VHkLSdlvT5jey7pvBf7',
                        'spotify:playlist:0sjYcaKmksfAcZVV9Zid3G',
                        'spotify:playlist:3GoE2qRab6zBwaO9r0b0nv',
                        'spotify:playlist:48IubOKEvNWeEZ1vECS0q3',
                        'spotify:playlist:3QGZ3W7vJbnnkWnbeh2j6M',
                        'spotify:playlist:5tmnS4UuyxxW41M7c3wDVB',
                        'spotify:playlist:0uBddqxhSNKw38usJ8j1y6',
                        'spotify:playlist:2QOeH7Fn7oU7pz0ZyL2cAq',
                        'spotify:playlist:4JuVgEhvl4WgNGk8nFvruv',
                        'spotify:playlist:1KIL5f54E1i2f4mSPTqu5B'
                       ]

In [340]:
playlists_dnb = ['spotify:playlist:068WHS0zOWsqvn2uIBYb5D',
                 'spotify:playlist:0Zarq4BVkFkZOWkmqsfrjA',
                 'spotify:playlist:3OYBpiQl7zOac5j5nGl3mu',
                 'spotify:playlist:4Xb7allH5pITXzHc1nadT7',
                 'spotify:playlist:37i9dQZF1DX1jlzMTQ2PY5',
                 'spotify:playlist:37i9dQZF1DX4to60X6goeK',
                 'spotify:playlist:5XGbuIRSb5INv66b817DJH',
                 'spotify:playlist:37i9dQZF1DX8jnAPF7Iiqp',
                 'spotify:playlist:5ABMzUESx7K7EyowE5kFCl',
                 'spotify:playlist:1K0VIne9ejeBhISlO43gRA',
                 'spotify:playlist:2vAT9MoHmHiOZ89zyk1uqd',
                 'spotify:playlist:37i9dQZF1DX5wDmLW735Yd',
                 'spotify:playlist:30sfJJYyp9raHP4HEYbACe',
                 'spotify:playlist:7CXXmPbwLrRJkr7EP6Fmrt',
                 'spotify:playlist:3gqEaRQUN0xYi9kHexWQpY',
                 'spotify:playlist:1CDdeNMmQ9jKQks8XD854G',
                 'spotify:playlist:19y0UVk0bcrJWEqMwBHosj',
                 'spotify:playlist:4oOZJEq1TBUti6PSouTo5M',
                 'spotify:playlist:0CjxEzL8Vm9IC3Or12KrAP',
                 'spotify:playlist:0oP76q6CyLk41eVDO0AxV1',
                 'spotify:playlist:05CWFSX7qOugRLcpy9VQED',
                 'spotify:playlist:6PwLxu5DoY39rGQiOq5GP2',
                 'spotify:playlist:4Oa99cnfYN2NmaUSTxiZaS',
                 'spotify:playlist:0W8KE1NZFTl8gpd7zHoQD2',
                 'spotify:playlist:0ydFX2Mwe0sX1bbHgma93g',
                 'spotify:playlist:1jnbdkFbVzMmDqUFnHSWid',
                 'spotify:playlist:2LP3pEcFPfVm15Pld7gwz0',
                 'spotify:playlist:5osGKmqCMhajgNmNIXAXmM',
                 'spotify:playlist:6yFDqUY09Hd3Ix3AamPYX4',
                 'spotify:playlist:5MG037sSypV015Ns1U6UwA',
                 'spotify:playlist:501tF4mo7f0rwwK8pJFSm6',
                 'spotify:playlist:6dDMrOGa5BJDDSbolmy9IC'
                ]

In [148]:
playlists_dubstep = ['spotify:playlist:3ObJ6Qra3CkV0gNCRTtK0c',
                     'spotify:playlist:6xo6Pr528QIucumzKcMXOu',
                     'spotify:playlist:37i9dQZF1DX4arVIN5Cg4U',
                     'spotify:playlist:6Qu7co7czjjXwSEuCyCOAD',
                     'spotify:playlist:5szak97F9g5KufvE9gI3wZ',
                     'spotify:playlist:2V0gyT0QmFdURZMO6V8uBC',
                     'spotify:playlist:2FACWZn19xOX3M8diykOQ9',
                     'spotify:playlist:37i9dQZF1DX5Q27plkaOQ3',
                     'spotify:playlist:0KEWfN1VI1gQIuZwnnyZrV',
                     'spotify:playlist:7r4NzA8aEKdNJcsLwRvNv8',
                     'spotify:playlist:1r3wxe2Bxdu5PE6qOkBPut',
                     'spotify:playlist:2ZBRrHz00yrqhCIUWpXgBI',
                     'spotify:playlist:5BLf2cCU57iKnVBYnXYZaO',
                     'spotify:playlist:3TMIAE4WfGH6NVo3FVJJke',
                     'spotify:playlist:09I0tyeHnd4G6vnt8iBYDM',
                     'spotify:playlist:3fRGztTdMolxw0nNRnZdj1',
                     'spotify:playlist:3quLyj3mybW3hHTt9UVGtw',
                     'spotify:playlist:2sjVpVZpcje4dIznYcQfAC',
                     'spotify:playlist:5iExAknaGP8JmScz4mdgp1',
                     'spotify:playlist:1AsVFc0487EvJwDqEIJHBa',
                     'spotify:playlist:5wNRJwSnBImUuZXJW1TiAj',
                     'spotify:playlist:3IA3ITmQpz3ljORSuLKI9g',
                     'spotify:playlist:1VBPcr2mWcjUBI89omHbq0',
                     'spotify:playlist:5XfRIAnUvliBwA78AfI21J',
                     'spotify:playlist:3vKsFlHqEkwbDLzUdm7k85',
                     'spotify:playlist:6E4ExdbRj89LWi61AOHjxo',
                     'spotify:playlist:6phIrRw2XU4sz3rYS7lSnv',
                     'spotify:playlist:45sWJkHfnQ3oJ1QxNiRSGN',
                     'spotify:playlist:2M4LqkvwUoZUTwaAfy2UKP',
                     'spotify:playlist:372Tvhb55xHYHlbGcwr2rv'
                    ]

In [149]:
playlists_hardstyle = ['spotify:playlist:3bGSAHGYFEDxyEj7uXe0qq',
                       'spotify:playlist:37i9dQZF1DX0pH2SQMRXnC',
                       'spotify:playlist:3M5bphHgTe5oTebqMUkczJ',
                       'spotify:playlist:4yHL0Vok6DofsmOEAUemTs',
                       'spotify:playlist:4FK8PNOzHWhZbgxvBIWxfr',
                       'spotify:playlist:2haltdKf2U7JhvCpeI3bpM',
                       'spotify:playlist:7ecjOhrAkcnWMJaJxMUiSK',
                       'spotify:playlist:3LHR9Ko2cIqfyOugDU6Sbr',
                       'spotify:playlist:37i9dQZF1DX7NhGf24haor',
                       'spotify:playlist:0jvp2XkmT8EAe8aKprzPSN',
                       'spotify:playlist:6jxElHkfE4i59rYA0iw7Nh',
                       'spotify:playlist:3TPVoqbqXyeYoiLr6f5Qzw',
                       'spotify:playlist:2MSHhxtmzMyBHjSVD0rcuN',
                       'spotify:playlist:2NY34sL3vaST2m41bbGj9I',
                       'spotify:playlist:4TGyWrMDsSEc1flS0XwCLR',
                       'spotify:playlist:0945N3eI9uOdY3dKnesoC8',
                       'spotify:playlist:2LVvgP7VJN5xAkmPdl5J4q',
                       'spotify:playlist:0J6rTAWgzATvixpbmoUyBw',
                       'spotify:playlist:3aR2n0XpRNlrWose8kx82S',
                       'spotify:playlist:2LPRY9zd2ou5nzBijiHhsT',
                       'spotify:playlist:02yGrg25tSif38IKdnu5BC',
                       'spotify:playlist:6yLZGml16kzULqUBjhWdKP',
                       'spotify:playlist:6bgAF9cZlt373xe3ZglWhv',
                       'spotify:playlist:1P6ue4O5NIFkfDgF9GFrkU',
                       'spotify:playlist:0r2Ah8Iq0F87jTIoSUPUFF',
                       'spotify:playlist:5hFd2Eakmovb6RAstuclNO',
                       'spotify:playlist:5SPe4piyussZvC5wS03gye',
                       'spotify:playlist:1QINLBaqnigRe2O5bnZrRS',
                       'spotify:playlist:2vGynBrYFZZahXI2yHrlua',
                       'spotify:playlist:3VdI2oR2OSTlggFuZf9DB4'
                      ]

In [150]:
playlists_prog_house = ['spotify:playlist:5CMvAWTlDPdZnkleiTHyyo',
                        'spotify:playlist:626UDFY53J9Fma3om0Jkx8',
                        'spotify:playlist:7LVIOFEgmTbDYnwGv162vN',
                        'spotify:playlist:5S33MuzhWWNW57z49eBSui',
                        'spotify:playlist:27VY0PivrvmGgNvFx23sU5',
                        'spotify:playlist:6JcBwspcgUBohikbK8vSzn',
                        'spotify:playlist:3YeajqIAyj4wpRF28r200x',
                        'spotify:playlist:5jYizoudbYsSmVmZvnkG7J',
                        'spotify:playlist:0twgzXVsIw5TTU3rTM35MS',
                        'spotify:playlist:05rymuboflZFqbz6qp2umN',
                        'spotify:playlist:3Wcdyxo643EZOQL1ES6M4s',
                        'spotify:playlist:4KRKrZA5Xd2Yg2DdqGkExt',
                        'spotify:playlist:2FfELJaBcybxCRkRJOv9HC',
                        'spotify:playlist:3aibqawnjfomPVR0Y8DaJX',
                        'spotify:playlist:6PCvHPLlCmRB40ecbX2ujS',
                        'spotify:playlist:5naf36PQw7BxiiTRBYudRv',
                        'spotify:playlist:6EH7SkbiK6hO5fo5HMedPX',
                        'spotify:playlist:528Hzmamt58Sjz2tVeiTyX',
                        'spotify:playlist:67Wb4vEhaQ8toEleqzHL2i',
                        'spotify:playlist:5M28rzdeQFlbMiymhMo1tC',
                        'spotify:playlist:3aIJD4EEfC26itf5RFttVs',
                        'spotify:playlist:5P1sC87GZ8F2d5zIo5NCJI',
                        'spotify:playlist:0YYfBqObHKDQgp03QgFpAF',
                        'spotify:playlist:77vGgwlmUAyaOcGouhK5X8',
                        'spotify:playlist:7bDn5o2sR24ZxpWHzhm2Qz',
                        'spotify:playlist:1R0QuM6uXZQRIojoxJK2Zj',
                        'spotify:playlist:2ATOfdwJgpYlkJ0zsexZYw',
                        'spotify:playlist:1oFzUnIGNXglnGfPuZefc5',
                        'spotify:playlist:0GVRyHxMZr6xjO0HoqilPe',
                        'spotify:playlist:4wdVFpxzUIr3nldYvt1DNg'
                       ]

In [192]:
playlists_techno = ['spotify:playlist:18vUeZ9BdtMRNV6gI8RnR6',
                    'spotify:playlist:37i9dQZF1DX6J5NfMJS675',
                    'spotify:playlist:6MJSGcF4iV79gyo8xZpd8U',
                    'spotify:playlist:3Y3NX4qhBo5rxvU60MgUoh',
                    'spotify:playlist:3xZiKQnlj2sG9s2mrQK1lZ',
                    'spotify:playlist:4C80pHNiCw3R7vcA5xvgPO',
                    'spotify:playlist:48yJRrbHtz1DwDEQEZuohc',
                    'spotify:playlist:2n9IW8aEnZ0C0o2e3etOdx',
                    'spotify:playlist:6JqrgEHEMAzgfyuQTXYReT',
                    'spotify:playlist:6VQ0Gn0ISUX4bAl9LVsQNK',
                    'spotify:playlist:5KlUhhSR7sZOdl8Hxy3Guz',
                    'spotify:playlist:37i9dQZF1DX8ZMwsPgxIOs',
                    'spotify:playlist:0B3WoheGNqol1B69LM9Y8n',
                    'spotify:playlist:0iDXvLH6Y6euU2bTxIjgxC',
                    'spotify:playlist:1irK0GrzY5R1ZTSKvD0egn',
                    'spotify:playlist:1wTBf9md3Qx1vEXJJ9VRpH',
                    'spotify:playlist:47km1wjUFyMh3LZ1J2R5D7',
                    'spotify:playlist:6sMkUAotBUAiXq1dP5s9nG',
                    'spotify:playlist:7LS2grTMmboABdGoy1Nbf4',
                    'spotify:playlist:7mwPa6HjqoiUrsk3C2Hitk',
                    'spotify:playlist:4JFZfMDsV30FAs3NR8y9Cq',
                    'spotify:playlist:7Anb1HtKdhvK3Pb1d36f22',
                    'spotify:playlist:3iSSSGjtYzad5IisyBrm2U',
                    'spotify:playlist:5qUiLFZRY0CdQJJNnvDMtY',
                    'spotify:playlist:0G2fxTnXZCwCfE5e3paeqq',
                    'spotify:playlist:7q0AoI7HnxTxfMpnH4Q92Z',
                    'spotify:playlist:1Ho106dv2ZNq07uVWFXFed',
                    'spotify:playlist:4teaXY1gkkq3Fv6ksOfmNC',
                    'spotify:playlist:7qNw8Sn2N5lbHYnirZIni6',
                    'spotify:playlist:7sCdZfCThcJRLjcQuV5qWe'
                   ]

In [193]:
playlists_trance = ['spotify:playlist:0Q3ugz23LAXFg2PvXJ8hMx',
                    'spotify:playlist:37i9dQZF1DXbtYAdenGE9U',
                    'spotify:playlist:69gyd1SXiVhvfsyYS078f0',
                    'spotify:playlist:78AFAJFvRzboZfEDnAkkFn',
                    'spotify:playlist:5dQ4RlPHRjGDZQwsWnpdJ2',
                    'spotify:playlist:1coHeAMbS3luasCXzfnm7b',
                    'spotify:playlist:1qxzbI2JYHwt1Uvcw6ZLPL',
                    'spotify:playlist:5VlGm8N7DJaMA7mCFOp6GV',
                    'spotify:playlist:0uVIGYfnUAkOT5REqtQICx',
                    'spotify:playlist:4wM86bbXb5U6IxOAXh5Wev',
                    'spotify:playlist:260cw4PvjDjcWuCi5duiEf',
                    'spotify:playlist:2iTMQ36cvjhxgIYGv7uld5',
                    'spotify:playlist:3dFiXOwSqPw3Qi59No6tfh',
                    'spotify:playlist:3TxOh9fFiyyOA8aTh7IaqC',
                    'spotify:playlist:12bgX5upoSMOSFr1HWpL02',
                    'spotify:playlist:5O4EoYxZlrkWJegYMMEMm3',
                    'spotify:playlist:7cEc9ewYHymDkAHvyQf344',
                    'spotify:playlist:3MKHH3b7ExUNwm228Za7wM',
                    'spotify:playlist:4nWDc1cjcL1yPkR3Z0uYeU',
                    'spotify:playlist:2Ps64TVbfmfX6jxWJnlX1j',
                    'spotify:playlist:1RX2XgpOYPAHPptQZZq5aF',
                    'spotify:playlist:2lrCh3HJpqVhx6Ia3HDdzm',
                    'spotify:playlist:5XZXBrQQ6D7aEjhJ2RC6Io',
                    'spotify:playlist:5qzG29K5RdtdFpvvpsMuY6',
                    'spotify:playlist:5QafFMGgQKGwqgV7k3qHy6',
                    'spotify:playlist:03JQeIhJa8jiLYLmwL7mBT',
                    'spotify:playlist:4U6C10Ui83IumPgqkvu36s',
                    'spotify:playlist:1t027lkuoGFJ6CQPs3WXSb',
                    'spotify:playlist:79BWXGe8jwjFr4vm6Md3qQ',
                    'spotify:playlist:72DnoMQTeGRgLbLk7WlKol'
                   ]

In [202]:
# Get track URI's from user generated playlists
track_uris_deep_house = tracks_from_playlists(playlists_deep_house, sp)
track_uris_dnb = tracks_from_playlists(playlists_dnb, sp)
track_uris_dubstep = tracks_from_playlists(playlists_dubstep, sp)
track_uris_hardstyle = tracks_from_playlists(playlists_hardstyle, sp)
track_uris_prog_house = tracks_from_playlists(playlists_prog_house, sp)
track_uris_techno = tracks_from_playlists(playlists_techno, sp)
track_uris_trance = tracks_from_playlists(playlists_trance, sp)

In [343]:
track_uris_dnb = tracks_from_playlists(playlists_dnb, sp)

In [ ]:
# Get audio features data for all tracks
audio_dnb = get_audio_features(track_uris_dnb, sp)
audio_deep_house = get_audio_features(track_uris_deep_house, sp)
audio_dubstep = get_audio_features(track_uris_dubstep, sp)
audio_hardstyle = get_audio_features(track_uris_hardstyle, sp)
audio_prog_house = get_audio_features(track_uris_prog_house, sp)
audio_techno = get_audio_features(track_uris_techno, sp)
audio_trance = get_audio_features(track_uris_trance, sp)

In [348]:
# Add genre tag and combine dataframes
audio_dnb['genre'] = 'dnb'
audio_deep_house['genre'] = 'deep_house'
audio_dubstep['genre'] = 'dubstep'
audio_hardstyle['genre'] = 'hardstyle'
audio_prog_house['genre'] = 'prog_house'
audio_techno['genre'] = 'techno'
audio_trance['genre'] = 'trance'

all_genres = pd.concat([audio_dnb, audio_deep_house, audio_dubstep, audio_hardstyle,
                        audio_prog_house, audio_techno, audio_trance])

In [349]:
print('Deep House Tracks:        ' + str(len(track_uris_deep_house)))
print('Drum and Bass Tracks:     ' + str(len(track_uris_dnb)))
print('Dubstep Tracks:           ' + str(len(track_uris_dubstep)))
print('Hardstyle Tracks:         ' + str(len(track_uris_hardstyle)))
print('Progressive House Tracks: ' + str(len(track_uris_prog_house)))
print('Techno Tracks:            ' + str(len(track_uris_techno)))
print('Trance Tracks:            ' + str(len(track_uris_trance)))

Deep House Tracks:        8960
Drum and Bass Tracks:     5014
Dubstep Tracks:           7403
Hardstyle Tracks:         6690
Progressive House Tracks: 14851
Techno Tracks:            7322
Trance Tracks:            9811


#### Spotify Recommendations

In [596]:
# Get Spotify track URI's for 1000 recommendations from each genre
recs_dnb = get_recommendations('drum-and-bass', 3000, 1000, sp)
recs_deep_house = get_recommendations('deep-house', 3000, 1000, sp)
recs_dubstep = get_recommendations('dubstep', 3000, 1000, sp)
recs_hardstyle = get_recommendations('hardstyle', 3000, 1000, sp)
recs_prog_house = get_recommendations('progressive-house', 3000, 1000, sp)
recs_techno = get_recommendations('techno', 3000, 1000, sp)
recs_trance = get_recommendations('trance', 3000, 1000, sp)

drum-and-bass: 999
deep-house: 1000
dubstep: 999
hardstyle: 998
progressive-house: 998
techno: 999
trance: 998


In [602]:
# Get audio features data for all recommended tracks
audio_dnb_recs = get_audio_features(recs_dnb, sp)
audio_deep_house_recs = get_audio_features(recs_deep_house, sp)
audio_dubstep_recs = get_audio_features(recs_dubstep, sp)
audio_hardstyle_recs = get_audio_features(recs_hardstyle, sp)
audio_prog_house_recs = get_audio_features(recs_prog_house, sp)
audio_techno_recs = get_audio_features(recs_techno, sp)
audio_trance_recs = get_audio_features(recs_trance, sp)

In [603]:
# Add genre tag and combine dataframes
audio_dnb_recs['genre'] = 'dnb'
audio_deep_house_recs['genre'] = 'deep_house'
audio_dubstep_recs['genre'] = 'dubstep'
audio_hardstyle_recs['genre'] = 'hardstyle'
audio_prog_house_recs['genre'] = 'prog_house'
audio_techno_recs['genre'] = 'techno'
audio_trance_recs['genre'] = 'trance'

all_genres_recs = pd.concat([audio_dnb_recs, audio_deep_house_recs, audio_dubstep_recs,
                             audio_hardstyle_recs, audio_prog_house_recs, audio_techno_recs,
                             audio_trance_recs])

#### Pickle the Raw Data

In [350]:
# Pickle the raw audio data
with open('pickle_raw_audio_data.pickle', 'wb') as to_write:
    pickle.dump(all_genres, to_write)

In [604]:
# Pickle the raw audio data
with open('pickle_raw_audio_data_recs.pickle', 'wb') as to_write:
    pickle.dump(all_genres_recs, to_write)